In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from vega_datasets import data

# Gradient Descent

This example follows the narrative shown in the [Gradient Descent, Step-by-Step](https://www.youtube.com/watch?v=sDv4f4s2SB8) video. Unlike the video, we will be using the cars dataset and computing values using Pandas. The video has really nice visualizations, so I recommend cross-referencing it for additional pointers.

We are going to select just the top N cars, which have the best MPG values. Since the video uses 3 data points, we'll select the top 3 cars.

In [ ]:
mpg = data('cars')
top5_mpg = mpg.sort_values('Miles_per_Gallon', ascending=False)[0:5]
top3_mpg = mpg.sort_values('Miles_per_Gallon', ascending=False)[0:3]

In [ ]:
#top5_mpg = top5_mpg.reset_index()
dataset = top3_mpg

In [ ]:
dataset[['Name', 'Miles_per_Gallon', 'Horsepower']]

In [ ]:
alt.Chart(dataset).mark_point().encode(
    x = "Horsepower",
    y = "Miles_per_Gallon"
)

Let's use Altair to find the regression line.

In [ ]:
chart = alt.Chart(dataset).mark_point().encode(
    alt.X("Horsepower", scale=alt.Scale(domain=(40, 70))),
    alt.Y("Miles_per_Gallon", scale=alt.Scale(domain=(40, 50)))
)

chart + chart.transform_regression('Horsepower', 'Miles_per_Gallon').mark_line(color="red").interactive()

Now that we have fit a line to our data, if someone tells us that their car has 58 horsepower, we can use the line to predict that they get 45 miles per gallon.

Let's see how the gradient descent can fit a line to data by finding the optimal values for the *intercept* and the *slope*.

$$Predicted\ MPG = intercept + slope * horsepower$$
or, mathematically,

$$ \hat y = \theta_0 + \theta_1 x$$

### Finding the intercept 

Let's first use **Gradient Descent** to find the intercept. Once we understand how it works, we'll use to solve for the intercept *and* the slope.

For now, let's plug-in the least squares estimate for the slope.

In [ ]:
mean_mpg = dataset['Miles_per_Gallon'].mean()
est_slope = np.mean(np.power(dataset['Miles_per_Gallon'] - mean_mpg, 2))
print(est_slope)

In [ ]:
slope = (45 - 44.4)/(58 - 48)
print(slope)

Let's pick a _random_ value for the intercept. This is just an initial guess that gives **Gradient Descent** something to improve on.

In this case, let's use 40, but technically, any number will work.

$$Predicted\ MPG = intercept + slope * horsepower$$
now becomes
$$Predicted\ MPG = 40 + slope * horsepower$$

It now gives us the equation for this line.

In [ ]:
est_intercept = 40

line_df = pd.DataFrame({
    'HP': dataset['Horsepower'],
    'est MPG': est_intercept + slope*dataset['Horsepower'].values
})
line_df

In [ ]:
chart + alt.Chart(line_df).mark_line(color="magenta").encode(
    alt.X('HP'),
    alt.Y('est MPG')
) + chart.transform_regression('Horsepower', 'Miles_per_Gallon').mark_line(color="red").interactive()

Let's evaluate how well this line fits the data using the **Sum of the Squared Residuals** (our Loss function). (*Note*: statistically speaking, [errors and residuals](https://en.wikipedia.org/wiki/Errors_and_residuals) are closely related but not the same.)

We can first compute the residual for the first car: that's Mazda GLC with 65 HP and the corresponding 46.6 MPG.

In [ ]:
dataset[['Name', 'Miles_per_Gallon', 'Horsepower']]

We get the predicted MPG, i.e., the point on the line, by plugging its HP into the equation of the line $$Predicted\ MPG = 40 + 0.06 * 65.0$$ to get the predicted value of 43.9.

In [ ]:
predicted_mpg = est_intercept + slope * 65 #dataset['Horsepower'].iloc[0]
predicted_mpg

The residual is the difference between the *observed MPG* ($y$) and the *predicted MPG* ($\hat y$).

In [ ]:
#dataset['Miles_per_Gallon'].iloc[0] - predicted_mpg
46.6 - predicted_mpg

In [ ]:
residuals = dataset['Miles_per_Gallon'] - (est_intercept + slope * dataset['Horsepower'])
residuals

The sum of the squared residuals (SSR)...

$$SSR = \sum (y - \hat y)^2 = \sum (y - (\theta_0 + \theta_1 x))^2$$

In [ ]:
ssr_40 = np.sum(np.power(residuals,2))
ssr_40

Now, we can plot the SSR for various intercepts.

In [ ]:
ssr = []

for intercept in range(40, 46):
    residuals = dataset['Miles_per_Gallon'] - (intercept + slope * dataset['Horsepower'])
    ssr.append(np.sum(np.power(residuals,2)))
    
ssr

In [ ]:
### Let's turn them into a dataframe for easy plotting
ssr_df = pd.DataFrame({
    'intercept': range(40, 46),
    'SSR': ssr
})
ssr_df

In [ ]:
alt.Chart(ssr_df).mark_circle().encode(
    alt.X('intercept:O'),
    alt.Y('SSR')
).interactive()

In order to visualize the different lines resulting from varying the intercept, let's create the dataframe that stores the results for different lines.

In [ ]:
lines_df = pd.DataFrame({
    'HP': dataset['Horsepower'],
    'MPG, i='+ str(est_intercept): est_intercept + slope*dataset['Horsepower'].values
})
lines_df

In [ ]:
for intercept in range(41, 46):
    column = 'MPG, i='+ str(intercept)
    lines_df[column] = intercept + slope*dataset['Horsepower'].values

In [ ]:
lines_df

In [ ]:
new_chart = chart + alt.Chart(lines_df).mark_line(color="magenta").encode(
        alt.X('HP'),
        alt.Y('MPG, i=40')
)
new_chart

In [ ]:
for intercept in range(40, 42): ### ADJUST the range values to see all lines
    column = 'MPG, i='+ str(intercept)
    new_chart = new_chart + alt.Chart(lines_df).mark_line(color="orange").encode(
        alt.X('HP'),
        alt.Y(column)
    )
new_chart # TODO: fix the y-axis labels :-))

Gradient descent identifies the optimal value by taking big steps when it is far away and smaller steps when it is close.

Remember plotting the SSR for the intercept estimates? We can obtain a line for the curve that goes through those points and we can take the derivative of this function to determine the slope at any value for the **intercept**.

$$SSR = \sum_i^N (y_i - \hat y_i)^2 = \sum_i^N (y_i - (\theta_0 + \theta_1 x_i))^2$$

Using the chain rule, we get:
$$\frac{\partial}{\partial \theta_0} = \sum_i^N -2(y_i - \theta_0 - \theta_1 x_i)$$

*Note: if we were using Linear Least Squares, we would solve for the optimal value of the intercept by finding where the slope of the line = 0.* 

In contrast, Gradient Descent finds the minimum value by taking steps from an initial guess (of the intercept) until it reaches the best value. This makes Gradient Descent very useful when it is not possible to solve for where the derivative = 0, and this is why it can be used in so many different situations.

In [ ]:
alt.Chart(ssr_df).mark_circle().encode(
    alt.X('intercept:O'),
    alt.Y('SSR')
).configure_axis(
    labelFontSize=14, # change axes label font size
    titleFontSize=16  # change axes title font size
).interactive()

In [ ]:
dt0 = -2* (dataset['Miles_per_Gallon'] - (est_intercept + slope * dataset['Horsepower']))
dt0.sum()

In [ ]:
est_intercept

So, when our intercept is 40, the slope of the curve (plotting the SSR against the intercept) is -9.4.

Note: The closer we get to the optimal value for the Intercept, the closer the slope of the curve gets to 0. This means that when the slope of the curve is close to 0, we should take baby steps, because we are close to the optimal value, and when the slope is far from 0, we should take big steps because we are far from the optimal value. 
However, if we take a huge step, then we would increase the SSR. So the size of the step should be related to the slope, since it tells us if we should take a baby step or a big step, but we need to make sure the step is not too big. Gradient Descent determines the **Step Size** by multiplying the slope by a small number called **The Learning Rate** ($\alpha$).

*Note: if the learning rate is too large, gradient descent might not arrive at the correct answer. In practice, a reasonable Learning Rate can be determined automatically by starting large and getting smaller with each step. In general, you should't need to worry about it too much.* 

In [ ]:
alpha = 0.1
dt0_res = dt0.sum()
step_size = dt0_res * alpha
step_size

With this step size, we can now calculate a new intercept. This allows us to take a closer step to the optimal value. Note how much the residuals would shrink with the new intercept.

In [ ]:
new_intercept = est_intercept - step_size
new_intercept

To take another step, we go back to the derivative and plug in a new intercept (40.94). That tells us a slope of the curve (-3.76).

In [ ]:
dt0 = -2* (dataset['Miles_per_Gallon'] - (new_intercept + slope * dataset['Horsepower']))
dt0.sum()

Let's calculate the step size.

In [ ]:
step_size1 = dt0.sum() * alpha
step_size1

Thus, the new intercept is ...

In [ ]:
new_intercept1 = new_intercept - step_size1
new_intercept1

Note that the SSR is getting smaller.

In [ ]:
# Using new_intercept
residuals = dataset['Miles_per_Gallon'] - (new_intercept + slope * dataset['Horsepower'])
np.sum(np.power(residuals,2))

In [ ]:
# Using the next intercept: new_intercept1
residuals = dataset['Miles_per_Gallon'] - (new_intercept1 + slope * dataset['Horsepower'])
np.sum(np.power(residuals,2))

Also note that the first step was larger than the second step.

In [ ]:
print("First step", round(step_size, 2))
print("Next step", round(step_size1, 2))

As we continue taking steps, each step gets smaller and smaller.

How does Gradient Descent know to stop taking steps?
Gradient Descent stops when the step size is very close to 0 (which will be when the slope is very close to 0).

> Step Size = Slope * Learning Rate

In practice, the minimum step size is 0.001 or smaller. 

However, we can also include a limit on the number of steps to take before giving up (e.g., 1000). So, even if the step size is still large, if we reach the maximum number of steps, Gradient Descent will stop.

### Let's review

The first thing we did is decide to use the Sum of the Squared Residuals (SSR) as the **Loss Function** to evaluate how well a line fits the data.

Then, we took the derivative of the SSR, i.e., the derivative of the loss function.

We picked a random value for the intercept, and **calculated the value of the derivative** using this randomly chosen intercept. This value told us (the direction and) **the size of the next step**.

We then calculated the **new intercept** as the difference between the old intercept and the step size.

Lastly, we plugged the new intercept into the derivative and repeated everything until Step Size was close to 0.

## Using Gradient Descent to find the slope _and_ the intercept 

When you have two or more derivatives of the same function, they are called **a gradient**.

We can now use this gradient to descent to the lowest point of the loss function.


$$SSR = \sum_i^N (y_i - \hat y_i)^2 = \sum_i^N (y_i - (\theta_0 + \theta_1 x_i))^2$$

Using the chain rule, we get two gradients (one for each coefficient):
$$\frac{\partial}{\partial \theta_0} = \sum_i^N -2(y_i - \theta_0 - \theta_1 x_i)$$

$$\frac{\partial}{\partial \theta_1} = \sum_i^N 2(y_i - \theta_0 - \theta_1 x_i) \times (-x_i)$$

In [ ]:
### Try to code up the algorithm yourself from start to finish
### Start with a fixing a slope and iteratively finding the intercept.
### Then, write a version that will find both, the slope and the intercept.